## Assignment 4

*100 points (8% of course grade)*</br>
*Assigned: Thu, Jun 27th*</br>
**Due: Mon, Jul 8th, 23:59**

This homework covers mongodb queries, and you may need to spend some time in setting up your MongoDB (will come back to Postgres in A5!). Some programming details are not covered in the lectures, so you will need to read the documentations and tutorials yourself. If you wait until the last minute, you might be overwhelmed.

You must turn in the required files electronically. Please follow the submission instructions at the end of this notebook.

### Problem 1 
Consider the information about the US congress stored as JSON documents in a MongoDB database. To start the MongoDB database server and construct this database named congress, follow the instructions in [MongoDB Tips](https://docs.google.com/document/d/1AaFotV4RMEnQTWAAqztMN5lsvsINPzCA6n7KB9GmZsE/edit?usp=sharing), and then use the following commands:

```bash
(enter your A4/congress folder)
mongorestore --gzip --archive=congress/mongodb-dump.tgz
```
The database contains two types of documents inside two collections, people and committees. To see these documents, use the following commands (warning: the entire database may be too long to print on your terminal):
```bash
mongosh congress --quiet --eval 'printjson(db.people.find().toArray())'
mongosh congress --quiet --eval 'printjson(db.committees.find().toArray())'
```
The structures of these documents are self-explanatory. The database has two collections.

- Each person in the `people` collection stores information about a legislator, including the roles they has served in the Congress. A role with type `rep` indicates a Representative (member of the House), while a role with type `sen` indicates a Senator (member of the Senate). A role is current if its `current` attribute equals 1.
- Each committee in the `committees` collection stores information about a committee. It has a list of members, whose ids reference those of people; `role` specifies the role of the member in the committee (e.g., chair or ranking member). Oftentimes a committee can have subcommittees. Each subcommittee element has its own list of members, which should be a subset of the committee members. A legislator can serve on multiple committees, and even multiple subcommittees under the same committee. 

Note that the documents are identified by their `_id` attribute values, which are person ids and committee codes, respectively.



Write MongoDB queries (in MongoDB shell syntax) to answer the following questions. Unless otherwise noted, please make sure that your answer appears as an array. Your query should have the form `printjson(db.collection.method(...).toArray())`, where collection is one of people and committees, and method is one of `find` and `aggregate`. For each question below, say (a), write your MongoDB query in the cell below the question. You can run your query directly in mongodb shell after you enter the interactive shell by running `mongosh` in your command line.
You may also run your query as follows by saving your query in a separate file:
```bash
mongosh congress --quiet -f a.js
```

See [MongoDB Tips](https://docs.google.com/document/d/1AaFotV4RMEnQTWAAqztMN5lsvsINPzCA6n7KB9GmZsE/edit?usp=sharing) for additional tips for MongoDB. The online autotester is available at https://ratest.cs.sfu.ca/mongo_test


#### a. (10 points) Find legislators with the exact last name of “Smith”. Simply print the entire person documents. 
You can use attr:/pattern/ to match the value of attr against a regular expression pattern. In particular, `/ XYZ$/` (note the space before XYZ) ensures that the string ends with a space followed by “XYZ”; `"$"` in the pattern matches the end of the string.


In [ ]:
/* input your answer in this cell: */
printjson(db.people.find({name: {$regex: /\S+ Smith/}}).toArray())

#### b. [12 points] Find who serves the role of "Ranking Member" for the House subcommittee on "Energy and Mineral Resources" (under House Committee HSII, "House Committee on Natural Resources"). Simply print the entire person document.


In [ ]:
printjson(db.committees.aggregate([
    { $match: { _id: "HSII" } },
    { $unwind: "$subcommittees" },
    { $match: { "subcommittees.displayname": "Energy and Mineral Resources" } },
    { $unwind: "$subcommittees.members" },
    { $match: { "subcommittees.members.role": "Ranking Member" } },
    { 
        $lookup: {
            from: "people",
            localField: "subcommittees.members.id",
            foreignField: "_id",
            as: "correctPerson"
        }
    },
    { $unwind: "$correctPerson" },
    { $replaceRoot: { newRoot: "$correctPerson" } }
]).toArray());

#### c. [12 points] List all current legislators born after the first version of Microsoft SQL Server released (April 24, 1989) but before the first version of Postgres released (July 8, 1996). Format each of them simply as {"name" : "...", "birthday": ...}. Order them by the name attribute. 

You can use `attr: { $gt: ISODate("2000-01-01") }` to test if the value of attr is later than the date 2000-01-01, or `attr: { $lt: ISODate("2000-01-01") }` to test if the value of attr is before the date 2000-01-01.

In [ ]:
/* input your answer in this cell: */
printjson(db.people.aggregate([
    {
        $match: {
            birthday: {
                $gt: ISODate('1989-04-24'),
                $lt: ISODate('1996-07-08')
            }
        }
    },
    {
        $sort: { name: 1 }
    },
    {
        $project: { name: 1, birthday: 1, _id: 0 }
    }
]).toArray());


#### d. [12 points] List all current female Democratic legislators. Format each of them as { "name": "…", "age": …, "state": "…", "type": "sen_or_rep" }, where age is an approximation obtained by subtracting their birth year from 2024. Order them by the age attribute (descending), with ties broken by name (ascending).

You can use `$year` to extract the year component from a date and $subtract to perform subtraction.


In [ ]:
/* input your answer in this cell: */
printjson(
    db.people.aggregate([
        { $match: { gender: "F" } },
        { $unwind: "$roles" },
        { $match: { "roles.current": 1, "roles.party": "Democrat" } },
        { 
            $project: {
                name: 1,
                age: { $subtract: [2024, { $year: "$birthday" }] },
                state: "$roles.state",
                type: "$roles.type",
                _id: 0
            }
        },
        { $sort: { age: -1, name: 1 } }
    ]).toArray()
);



#### e. [12 points] List the name, district, and party of each current Representative of WA. Format each of them in the form { "name": "…", "district": …, "party": "…" } and sort them according to the district.


In [ ]:
/* input your answer in this cell: */
printjson(
    db.people.aggregate([
        { $unwind: "$roles" },
        { $match: { "roles.state": "WA", "roles.current": 1, "roles.type": "rep" } },
        { $project: { name: 1, district: "$roles.district", party: "$roles.party", _id: 0 } },
        { $sort: { district: 1 } }
    ]).toArray()
);

#### f. [12 points] List the names of current Senators who at some point earlier also served as Representatives. Format each of them as in the form { "name": "…" }. Order them by name (ascending).


In [ ]:
/* input your answer in this cell: */
printjson(
    db.people.aggregate([
        { $match: { "roles.type": "rep" } },
        { $unwind: "$roles" },
        { $match: { "roles.type": "sen", "roles.current": 1 } },
        { $group: { _id: "$name" } },
        { $project: { name: "$_id", _id: 0 } },
        { $sort: { name: 1 } }
    ]).toArray()
);



#### g. [15 points] List the names of legislators who are currently serving New York but NOT serving in any committee or subcommittee. Format each of them of an element of the form { "name": "…" }. Order them by name (ascending).


In [ ]:
/* input your answer in this cell: */
printjson(
    db.people.aggregate([
        { $unwind: "$roles" },
        { $match: { "roles.state": "NY", "roles.current": 1 } },
        { 
            $lookup: {
                from: "committees",
                localField: "_id",
                foreignField: "members.id",
                as: "served_in"
            }
        },
        { $match: { served_in: { $eq: [] } } },
        { $sort: { name: 1 } },
        { $project: { name: 1, _id: 0 } }
    ]).toArray()
);


#### h. [15 points] Find the number of current legislators for each state and territory (both are stored in the state attribute) by gender. Sort the states/territories by name (two-letter abbreviation) alphabetically. Your output should look like the following (whitespace is unimportant):
[
  { state: 'AK', M: 1, F: 2 },
  { state: 'AL', M: 7, F: 2 },
  { state: 'AR', M: 6, F: 0 },
  { state: 'AS', M: 0, F: 1 },
…]


In [ ]:
/* input your answer in this cell: */
printjson(
    db.people.aggregate([
        { $unwind: "$roles" },
        { $match: { "roles.current": 1 } },
        { $group: { _id: { state: "$roles.state", gender: "$gender" }, count: { $sum: 1 } } },
        { 
            $group: {
                _id: "$_id.state",
                M: {
                    $sum: {
                        $cond: [{ $eq: ["$_id.gender", "M"] }, "$count", 0]
                    }
                },
                F: {
                    $sum: {
                        $cond: [{ $eq: ["$_id.gender", "F"] }, "$count", 0]
                    }
                }
            }
        },
        { $sort: { _id: 1 } },
        { $project: { state: "$_id", M: 1, F: 1, _id: 0 } }
    ]).toArray()
);
